# 03 - Federated Learning


Define the available types of federated learning.

- 'STRATIFIED': Stratified sampling of the data. The data is split into a number of shards, and each shard is assigned to a client. The data is split in a stratified manner, meaning that the distribution of the labels is approximately the same in each shard.
- 'LEAVE_ONE_OUT' - Each client is assigned a shard of data, each shard is missing one of the attack labels. Other clients in the network are exposed to the attack label, but the specific client is not. This demonstrates the ability of federated learning to protect against unknown attacks.
- 'ONE_CLASS' - Each client is assigned a shard of data, each shard contains only one of the attack labels.
- 'HALF_BENIGN' - Half of the clients are exposed to Benign data only, the other half are exposed to all data.


In [94]:
### THIS SECTION NEEDS TO BE SET TO DETERMINE WHICH CONFIGURATION METHOD TO UTILISE

SPLIT_AVAILABLE_METHODS = ['STRATIFIED','LEAVE_ONE_OUT', 'ONE_CLASS', 'HALF_BENIGN' ]
METHOD = 'LEAVE_ONE_OUT'
NUM_OF_STRATIFIED_CLIENTS = 10  # only applies to stratified method
NUM_OF_ROUNDS = 10              # Number of FL rounds


The above test method in conjunction with the below classification selection will determine the number of clients.

EG:
`STRATIFIED` with:

- `ALL TYPES` - Results in `NUM_OF_STRATIFIED_CLIENTS` clients. Each client will have a stratified sample of the data.

`LEAVE_ONE_OUT` with:

- `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 32 attack labels.
- `group_classifier` - Results in 7 clients. Each client will have benign traffic and 6 attack groups.
- `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels.

`ONE_CLASS` with:

- `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 1 attack label.
- `group_classifier` - Results in 7 clients. Each client will have benign traffic and 1 attack groups.
- `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS LEAVE_ONE_OUT for binary classifier

`HALF_BENIGN` with:

- `individual_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 33 malicious attack labels.
- `group_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 7 malicious attack groups.
- `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS LEAVE_ONE_OUT for binary classifier


In [95]:
individual_classifier = False
group_classifier = False
binary_classifier = True


Include the defines for the dataframe columns and the attack labels and their mappings


In [96]:
# from google.colab import drive
# drive.mount('/content/drive')

In [97]:
# import sys
# sys.path.append('/content/drive/MyDrive/Do_An_Tot_Nghiep/CICIoT2023/')

In [98]:
from enum import Enum
from includes import *


In [99]:
# Gỡ các thư viện (nếu còn sót)
# !pip3 uninstall -y numpy tensorflow torch torchvision flwr scikit-learn

# Cài numpy trước để tránh lỗi tương thích
!pip3 install numpy==1.22.4

# Cài các thư viện còn lại
!pip3 install tensorflow==2.11.0
!pip3 install torch==2.0.1 torchvision==0.15.2
!pip3 install "flwr[simulation]==1.5.0"
!pip3 install scikit-learn==1.2.2
!pip3 install pandas matplotlib

Python(76169) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


  Using cached numpy-1.22.4-cp310-cp310-macosx_10_15_x86_64.whl.metadata (2.0 kB)
Using cached numpy-1.22.4-cp310-cp310-macosx_10_15_x86_64.whl (17.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.3.2 requires numpy>=1.23, but you have numpy 1.22.4 which is incompatible.
matplotlib 3.10.3 requires numpy>=1.23, but you have numpy 1.22.4 which is incompatible.
scipy 1.15.3 requires numpy<2.5,>=1.23.5, but you have numpy 1.22.4 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


Python(76171) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


  Using cached protobuf-3.19.6-py2.py3-none-any.whl.metadata (828 bytes)
Using cached protobuf-3.19.6-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googleapis-common-protos 1.70.0 requires protobuf!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.


Python(76176) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(76178) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


Python(76181) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


  Using cached numpy-2.2.6-cp310-cp310-macosx_14_0_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp310-cp310-macosx_14_0_x86_64.whl (6.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flwr 1.5.0 requires numpy<2.0.0,>=1.21.0, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


Python(76185) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


## Imports


In [100]:
%pip install numpy tensorflow torch flwr

import numpy as np
import tensorflow as tf
import torch
import flwr as fl

print("NumPy version:", np.__version__)
print("TensorFlow version:", tf.__version__)
print("PyTorch version:", torch.__version__)
print("Flower version:", fl.__version__)

Python(76187) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
NumPy version: 1.26.4
TensorFlow version: 2.16.2
PyTorch version: 2.2.2
Flower version: 1.18.0


In [101]:
%pip install pandas

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split

Python(76193) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [102]:
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.18.0
numpy 1.26.4
torch 2.2.2
Training on cpu


## Load the Dataset


## Training data


Either read the training pickle file if it exists, or process the dataset from scratch.


In [ ]:
import pandas as pd
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

DATASET_DIRECTORY = './Data/'

# Kiểm tra xem tệp 'training_data.pkl' có tồn tại không
if os.path.isfile('training_data.pkl'):
    print("File exists, loading data...")
    train_df = pd.read_pickle('training_data.pkl')
    print("Training data loaded from pickle file.")
else:
    # Lấy danh sách các tệp CSV trong thư mục
    try:
        df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
        df_sets.sort()
    except FileNotFoundError:
        print(f"Directory not found: {DATASET_DIRECTORY}")
        raise

    # Chia tập huấn luyện (80%) và tập kiểm tra (20%)
    training_sets = df_sets[:int(len(df_sets) * 0.8)]
    test_sets = df_sets[int(len(df_sets) * 0.8):]

    # In số lượng tệp
    print('Training sets: {}'.format(len(training_sets)))
    print('Test sets: {}'.format(len(test_sets)))

    # Kiểm tra xem training_sets có rỗng không
    if not training_sets:
        print("No CSV files found in the directory. Please check DATASET_DIRECTORY.")
        raise FileNotFoundError("No CSV files found.")

    # Đọc và nối các tập huấn luyện
    dfs = []
    print("Reading training data...")
    for train_set in tqdm(training_sets):
        file_path = os.path.join(DATASET_DIRECTORY, train_set)  # Sử dụng os.path.join để xử lý đường dẫn
        try:
            df_new = pd.read_csv(file_path)
            dfs.append(df_new)
        except FileNotFoundError:
            print(f"File not found: {file_path}. Skipping...")
            continue
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}. Skipping...")
            continue

    # Kiểm tra xem có dữ liệu nào được đọc không
    if not dfs:
        print("No valid CSV files were loaded. Please check the files in the directory.")
        raise ValueError("No data loaded.")

    # Nối các dataframe
    train_df = pd.concat(dfs, ignore_index=True)
    print("Complete training data size: {}".format(train_df.shape))

    # Ánh xạ cột 'label' sang dict_34_classes (giả định đã được định nghĩa)
    try:
        train_df['label'] = train_df['label'].map(dict_34_classes)
    except NameError:
        print("Error: dict_34_classes is not defined. Please define the mapping dictionary.")
        raise

    # Chia tập huấn luyện với tỉ lệ TRAIN_SIZE
    TRAIN_SIZE = 0.05
    print(f"Splitting the data into {TRAIN_SIZE * 100}%")

    try:
        X_train, X_test, y_train, y_test = train_test_split(
            train_df[X_columns], train_df[y_column],
            test_size=(1 - TRAIN_SIZE), random_state=42, stratify=train_df[y_column]
        )
    except NameError:
        print("Error: X_columns or y_column is not defined. Please define these variables.")
        raise

    # Kết hợp lại X_train và y_train
    train_df = pd.concat([X_train, y_train], axis=1)

    # Dọn dẹp biến không dùng
    del X_train, y_train, X_test, y_test

    # Lưu dữ liệu vào tệp pickle
    print("Writing training data to pickle file...")
    try:
        train_df.to_pickle('training_data.pkl')
    except Exception as e:
        print(f"Error saving pickle file: {str(e)}")

# In kích thước tập huấn luyện cuối cùng
print("Training data size: {}".format(train_df.shape))

File exists, loading data...
Training data loaded from pickle file.
Training data size: (94650, 47)


In [104]:
# show the unique values counts in the label column for train_df
print("Counts of attacks in train_df:")
print(train_df['label'].value_counts())

Counts of attacks in train_df:
label
6     14591
4     10974
5      9154
2      8300
3      8250
1      8185
7      7270
13     6754
15     5404
14     4080
0      2226
17     1995
19     1812
18     1520
10      919
26      624
9       586
8       581
25      369
24      274
21      202
22      171
16      150
23       75
12       59
11       47
33       27
27       12
31       10
32        9
28        7
29        6
20        4
30        3
Name: count, dtype: int64


In [105]:
train_df.describe()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
count,94650.000000,9.465000e+04,94650.000000,94650.000000,9.465000e+04,9.465000e+04,9.465000e+04,94650.000000,94650.000000,94650.000000,...,94650.000000,94650.000000,9.465000e+04,94650.000000,94650.000000,94650.000000,9.465000e+04,94650.000000,94650.000000,94650.000000
mean,5.071891,7.661711e+04,9.061305,66.300328,9.432170e+03,9.432170e+03,9.794633e-07,0.086445,0.206878,0.090343,...,32.920071,125.303607,8.320343e+07,9.499373,13.122775,46.521285,2.970717e+04,0.096555,141.536086,7.038204
std,244.630430,4.610147e+05,8.924526,13.756123,1.041630e+05,1.041630e+05,1.320695e-04,0.281021,0.405069,0.286675,...,157.296600,244.338380,1.706835e+07,0.820194,8.657428,222.473009,3.118869e+05,0.232961,21.093922,5.550426
min,0.000000,0.000000e+00,0.600000,6.400000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,42.000000,9.536743e-07,1.000000,9.165151,0.000000,0.000000e+00,0.000000,1.000000,0.000000
25%,0.000000,5.400000e+01,6.000000,64.000000,2.111600e+00,2.111600e+00,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,50.000000,8.307156e+07,9.500000,10.000000,0.000000,0.000000e+00,0.000000,141.550000,3.000000
50%,0.000000,5.400000e+01,6.000000,64.000000,1.591258e+01,1.591258e+01,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,54.000000,8.312452e+07,9.500000,10.392305,0.000000,0.000000e+00,0.000000,141.550000,6.000000
75%,0.102921,3.202000e+02,14.440000,64.000000,1.189609e+02,1.189609e+02,0.000000e+00,0.000000,0.000000,0.000000,...,0.372329,54.060000,8.334391e+07,9.500000,10.397381,0.505921,1.377858e+00,0.080000,141.550000,10.000000
max,51182.926771,9.760969e+06,47.000000,255.000000,6.291456e+06,6.291456e+06,2.964676e-02,1.000000,1.000000,1.000000,...,6364.598485,4989.200000,1.676394e+08,13.500000,113.212734,9000.901497,4.534864e+07,1.000000,244.600000,33.000000


In [106]:
train_df

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
1761622,0.000000,54.00,6.00,67.82,82.860270,82.860270,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.331437e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,2
1576787,4.371373,111.52,6.11,64.00,0.457582,0.457582,0.0,0.0,1.0,0.0,...,10.484298,55.76,8.297270e+07,9.5,10.776277,14.840357,620.739807,0.19,141.55,14
1193126,0.118171,31529.50,17.00,64.00,7888.457914,7888.457914,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.309880e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,4
256506,0.001054,54.86,6.00,64.00,4.084927,4.084927,0.0,0.0,0.0,0.0,...,0.654838,54.12,8.333098e+07,9.5,10.416341,0.926990,2.525671,0.18,141.55,2
524927,0.000393,1.74,45.24,64.40,48.935368,48.935368,0.0,0.0,0.0,0.0,...,43.566367,557.24,8.364703e+07,9.5,33.590057,61.128964,6279.270791,0.31,141.55,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1733004,0.000000,162.49,17.00,64.59,26.812572,26.812572,0.0,0.0,0.0,0.0,...,1.518631,162.49,8.300767e+07,9.5,18.023884,2.151377,19.532203,0.12,141.55,13
1491919,0.000000,54.00,6.00,64.00,1.173496,1.173496,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.333114e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,2
1363686,0.000000,54.00,6.00,64.00,24.342747,24.342747,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.331373e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,2
42579,0.000000,81.00,6.00,64.00,6.689647,6.689647,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.292582e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,15


---

## Test Data

Concat the test data into a single dataframe


In [107]:
import pickle

# Check if the pickle file exists
testing_data_pickle_file = 'testing_data.pkl'

try:
    # Try to load the pickle file
    test_df = pd.read_pickle(testing_data_pickle_file)
    print("Test data loaded from pickle file.")
except (FileNotFoundError, EOFError, pickle.UnpicklingError) as e:
    # Handle both file not found and pickle loading errors
    print(f"No file find or error loading pickle file testing_data.pkl")
    
    # Get list of CSV files and sort them
    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    test_sets = df_sets[int(len(df_sets) * 0.8):]

    # Read and concatenate test data
    dfs = []
    print("Reading test data...")
    for test_set in tqdm(test_sets):
        try:
            df_new = pd.read_csv(os.path.join(DATASET_DIRECTORY, test_set))
            dfs.append(df_new)
        except FileNotFoundError:
            print(f"File not found: {test_set}. Skipping...")
        except Exception as e:
            print(f"Error reading {test_set}: {e}. Skipping...")
    
    # Concatenate all dataframes
    test_df = pd.concat(dfs, ignore_index=True)

    # Map labels to integers 
    try:
        test_df['label'] = test_df['label'].map(dict_34_classes)
    except KeyError as e:
        print(f"Error mapping labels: {e}. Check dict_34_classes.")
        raise

    # Save the test data to pickle file
    try:
        test_df.to_pickle(testing_data_pickle_file)
        print(f"Test data saved to {testing_data_pickle_file}.")
    except Exception as e:
        print(f"Error saving pickle file: {e}")

print("Testing data size: {}".format(test_df.shape))

Test data loaded from pickle file.
Testing data size: (725335, 47)


In [108]:
print("Number of rows in train_df: {}".format(len(train_df)))
print("Number of rows in test_df: {}".format(len(test_df)))

train_size = len(train_df)
test_size = len(test_df)

Number of rows in train_df: 94650
Number of rows in test_df: 725335


---

# Scale the test and train data


### Scale the training data input features


In [109]:
X_columns = [col for col in train_df.columns if col != 'label']
y_column = 'label'
##
scaler = StandardScaler()
train_df[X_columns] = scaler.fit_transform(train_df[X_columns])

### Scale the testing data input features


In [110]:
test_df[X_columns] = scaler.fit_transform(test_df[X_columns])

---

# Define the classification problem - (2 classes, 8 classes or 34 classes)

Change the following cell to select the classification type

If the METHOD == STRATIFIED, then we can use any classifier
If the METHOD == ATTACK_GROUP then we must use Group Classifier.


In [111]:

class_size_map = {2: "Binary", 8: "Group", 34: "Individual"}

if group_classifier:
    print("Group 8 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_7_classes values
    test_df['label'] = test_df['label'].map(dict_8_classes)
    train_df['label'] = train_df['label'].map(dict_8_classes)
    class_size = "8"

elif binary_classifier:
    print("Binary 2 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_2_classes values
    test_df['label'] = test_df['label'].map(dict_2_classes)
    train_df['label'] = train_df['label'].map(dict_2_classes)
    class_size = "2"

else:
    print ("Individual 34 Class classifier... - No adjustments to labels in test and train dataframes")
    class_size = "34"


Binary 2 Class Classifier... - Adjusting labels in test and train dataframes


---

# Split the Training Data into partitions for the Federated Learning clients depending on the test required

As a reminder:

`STRATIFIED` with:

- `ALL TYPES` - Results in `NUM_OF_STRATIFIED_CLIENTS` clients. Each client will have a stratified sample of the data.

`LEAVE_ONE_OUT` with:

- `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 32 attack labels.
- `group_classifier` - Results in 7 clients. Each client will have benign traffic and 6 attack groups.
- `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels.

`ONE_CLASS` with:

- `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 1 attack label.
- `group_classifier` - Results in 7 clients. Each client will have benign traffic and 1 attack groups.
- `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS LEAVE_ONE_OUT for binary classifier

`HALF_BENIGN` with:

- `individual_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 33 malicious attack labels.
- `group_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 7 malicious attack groups.
- `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS LEAVE_ONE_OUT for binary classifier


In [112]:
from sklearn.model_selection import StratifiedKFold

# Define fl_X_train and fl_y_train
fl_X_train = []
fl_y_train = []

client_df = pd.DataFrame()

if METHOD == 'STRATIFIED':
    print(f"{Colours.YELLOW.value}STRATIFIED METHOD{Colours.NORMAL.value} with {class_size} class classifier")
    # We are going to split the training data into 'NUM_OF_STRATIFIED_CLIENTS' smaller groups using StratifiedKFold
    skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(train_df[X_columns], train_df[y_column]):
        fl_X_train.append(train_df[X_columns].iloc[test_index])
        fl_y_train.append(train_df[y_column].iloc[test_index])

elif METHOD == 'LEAVE_ONE_OUT':
    print(f"{Colours.YELLOW.value}LEAVE_ONE_OUT METHOD{Colours.NORMAL.value} with {class_size} class classifier")

    if individual_classifier or group_classifier:
        # Set the number of splits required to the number of classes - 1
        num_splits = int(class_size) - 1
    else:
        # For binary classifier, set the number of splits to 10
        num_splits = 10

    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    # When creating the clients, we will remove one attack class from the training data
    # For the binary classifier, evey other client will have the benign class removed
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if binary_classifier:
            print(f"i: {i} = i % 2 = {i % 2}")
            if i % 2 == 0:
                print("Benign only")
                # Create a new dataframe for the client data with only benign traffic
                client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
                fl_X_train.append(client_df[X_columns])
                fl_y_train.append(client_df[y_column])
            else:
                print("Both")
                # Create a new dataframe for the client data
                fl_X_train.append(train_df[X_columns].iloc[test_index])
                fl_y_train.append(train_df[y_column].iloc[test_index])
        else:
            # Create a new dataframe for the client data
            client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i+1]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])

elif METHOD == 'ONE_CLASS':
    print(f"{Colours.YELLOW.value}ONE_CLASS METHOD{Colours.NORMAL.value} with {class_size} class classifier")
    # Each client only has one attack class in their training data along with the Benign data

    if individual_classifier or group_classifier:
        # Set the number of splits required to the number of classes - 1
        num_splits = int(class_size) - 1
    else:
        # For binary classifier, set the number of splits to 10
        num_splits = 10

    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    # When creating the clients, we will only add the benign data and the attack class for that client
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if binary_classifier:
            print(f"i: {i} = i % 2 = {i % 2}")
            if i % 2 == 0:
                print("Benign only")
                # Create a new dataframe for the client data with only benign traffic
                client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
                fl_X_train.append(client_df[X_columns])
                fl_y_train.append(client_df[y_column])
            else:
                print("Both")
                # Create a new dataframe for the client data
                fl_X_train.append(train_df[X_columns].iloc[test_index])
                fl_y_train.append(train_df[y_column].iloc[test_index])
        else:
            # Create a new dataframe for the client data
            client_df = pd.concat([train_df.iloc[test_index][(train_df[y_column] == 0) | (train_df[y_column] == i+1)]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])

elif METHOD == 'HALF_BENIGN':
    print(f"{Colours.YELLOW.value}HALF_BENIGN METHOD{Colours.NORMAL.value} with {class_size} class classifier")

    num_splits = 10

    # Split into 10 client data
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    # For i % 2 == 0, add only benign data
    # For i % 2 == 1, add all data
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if i % 2 == 0:
            print("Benign only")
            # Create a new dataframe for the client data with only benign traffic
            client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] == 0]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])
        else:
            print("All Classes")
            fl_X_train.append(train_df[X_columns].iloc[test_index])
            fl_y_train.append(train_df[y_column].iloc[test_index])
else:
    print(f"{Colours.RED.value}ERROR: Method {METHOD} not recognised{Colours.NORMAL.value}")
    exit()



LEAVE_ONE_OUT METHOD with 2 class classifier
i: 0 = i % 2 = 0
Benign only
i: 1 = i % 2 = 1
Both
i: 2 = i % 2 = 0
Benign only
i: 3 = i % 2 = 1
Both
i: 4 = i % 2 = 0
Benign only
i: 5 = i % 2 = 1
Both
i: 6 = i % 2 = 0
Benign only
i: 7 = i % 2 = 1
Both
i: 8 = i % 2 = 0
Benign only
i: 9 = i % 2 = 1
Both


In [113]:
NUM_OF_CLIENTS = len(fl_X_train)

for i in range(len(fl_X_train)):
    # Show the unique values in the y column
    (f"Client ID: {i}")
    print(f"fl_X_train[{i}].shape: {fl_X_train[i].shape}")
    print(f"fl_y_train[{i}].value_counts():\n{fl_y_train[i].value_counts()}")
    print(f"fl_y_train[{i}].unique(): {fl_y_train[i].unique()}\n")

# Check that fl_X_train[0] and fl_X_train[1] contain different data
print(f"fl_X_train[0].equals(fl_X_train[1]): {fl_X_train[0].equals(fl_X_train[1])}")

fl_X_train[0].shape: (222, 46)
fl_y_train[0].value_counts():
label
0    222
Name: count, dtype: int64
fl_y_train[0].unique(): [0]

fl_X_train[1].shape: (9465, 46)
fl_y_train[1].value_counts():
label
1    9243
0     222
Name: count, dtype: int64
fl_y_train[1].unique(): [1 0]

fl_X_train[2].shape: (222, 46)
fl_y_train[2].value_counts():
label
0    222
Name: count, dtype: int64
fl_y_train[2].unique(): [0]

fl_X_train[3].shape: (9465, 46)
fl_y_train[3].value_counts():
label
1    9243
0     222
Name: count, dtype: int64
fl_y_train[3].unique(): [1 0]

fl_X_train[4].shape: (223, 46)
fl_y_train[4].value_counts():
label
0    223
Name: count, dtype: int64
fl_y_train[4].unique(): [0]

fl_X_train[5].shape: (9465, 46)
fl_y_train[5].value_counts():
label
1    9242
0     223
Name: count, dtype: int64
fl_y_train[5].unique(): [1 0]

fl_X_train[6].shape: (223, 46)
fl_y_train[6].value_counts():
label
0    223
Name: count, dtype: int64
fl_y_train[6].unique(): [0]

fl_X_train[7].shape: (9465, 46)
fl_y_trai

Prepare an output directory where we can store the results of the federated learning


In [114]:
# Create an "Output" directory if it doesnt exist already
if not os.path.exists("Output"):
    os.makedirs("Output")

sub_dir_name = f"train_size-{train_size}"

# if sub_dir_name does not exist, create it
if not os.path.exists(f"Output/{sub_dir_name}"):
    os.makedirs(f"Output/{sub_dir_name}")

test_directory_name = f"{METHOD}_Classifier-{class_size}_Clients-{NUM_OF_CLIENTS}"

# Create an "Output/{METHOD}-{NUM_OF_CLIENTS}-{NUM_OF_ROUNDS}" directory if it doesnt exist already
if not os.path.exists(f"Output/{sub_dir_name}/{test_directory_name}"):
    os.makedirs(f"Output/{sub_dir_name}/{test_directory_name}")

# Ensure the directory is empty
for file in os.listdir(f"Output/{sub_dir_name}/{test_directory_name}"):
    file_path = os.path.join(f"Output/{sub_dir_name}/{test_directory_name}", file)
    if os.path.isfile(file_path):
        os.unlink(file_path)

# Original training size is the sum of all the fl_X_train sizes
original_training_size = 0
for i in range(len(fl_X_train)):
    original_training_size += fl_X_train[i].shape[0]

# Write this same info to the output directory/Class Split Info.txt
with open(f"Output/{sub_dir_name}/{test_directory_name}/Class Split Info.txt", "w") as f:
    for i in range(len(fl_X_train)):
        f.write(f"Client ID: {i}\n")
        f.write(f"fl_X_train.shape: {fl_X_train[i].shape}\n")
        f.write(f"Training data used {original_training_size}")
        f.write(f"fl_y_train.value_counts():\n{fl_y_train[i].value_counts()}\n")
        f.write(f"fl_y_train.unique(): {fl_y_train[i].unique()}\n\n")

### Convert the training dataset


In [115]:
# Convert the testing daya to X_test and y_test ndarrays
X_test = test_df[X_columns].to_numpy()
y_test = test_df[y_column].to_numpy()

In [116]:
num_unique_classes = len(train_df[y_column].unique())

train_df_shape = train_df.shape
test_df_shape = test_df.shape

# We are now done with the train_df and test_df dataframes, so we can delete them to free up memory
del train_df
del test_df
del client_df

---

### Data check


In [117]:
print("NUM_CLIENTS:", NUM_OF_CLIENTS)

print("NUM_ROUNDS:", NUM_OF_ROUNDS)
print()


print("Original training size: {}".format(original_training_size))


print("Checking training data split groups")
for i in range(len(fl_X_train)):
    print(i, ":", "X Shape", fl_X_train[i].shape, "Y Shape", fl_y_train[i].shape)


# Print the sizes of X_test and y_test
print("\nChecking testing data")
print("X_test size: {}".format(X_test.shape))
print("y_test size: {}".format(y_test.shape))

print("\nDeploy Simulation")

NUM_CLIENTS: 10
NUM_ROUNDS: 10

Original training size: 48438
Checking training data split groups
0 : X Shape (222, 46) Y Shape (222,)
1 : X Shape (9465, 46) Y Shape (9465,)
2 : X Shape (222, 46) Y Shape (222,)
3 : X Shape (9465, 46) Y Shape (9465,)
4 : X Shape (223, 46) Y Shape (223,)
5 : X Shape (9465, 46) Y Shape (9465,)
6 : X Shape (223, 46) Y Shape (223,)
7 : X Shape (9465, 46) Y Shape (9465,)
8 : X Shape (223, 46) Y Shape (223,)
9 : X Shape (9465, 46) Y Shape (9465,)

Checking testing data
X_test size: (725335, 46)
y_test size: (725335,)

Deploy Simulation


---

# Federated Learning

## Import the libraries and print the versions


In [118]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout



Define the Client and Server code


In [119]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

print('scikit-learn {}.'.format(sklearn.__version__))
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

import datetime

client_evaluations = []

class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=5, batch_size=32)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, self.train_labels, batch_size=32)
        print(f"{Colours.YELLOW.value}Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}{Colours.NORMAL.value}")

        # Write the same message to the "Output/{cid}_Evaluation.txt" file
        with open(f"Output/{sub_dir_name}/{test_directory_name}/{self.cid}_Evaluation.txt", "a") as f:
            f.write(f"{datetime.datetime.now()} - Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}\n")

            # Close the file
            f.close()

        return loss, len(self.train_data), {"accuracy": accuracy}

    def predict(self, incoming):
        prediction = np.argmax( self.model.predict(incoming) ,axis=1)
        return prediction

def client_fn(cid: str) -> NumpyFlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(50, activation='relu'),
      Dense(25, activation='relu'),
      Dense(num_unique_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c)


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
eval_count = 0

def get_evaluate_fn(server_model):
    global eval_count
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round


    def evaluate(server_round, parameters, config):
        global eval_count

        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print (f"Server Evaluating... Evaluation Count:{eval_count}")
        loss, accuracy = server_model.evaluate(X_test, y_test)

        y_pred = server_model.predict(X_test)
        print ("Prediction: ", y_pred, y_pred.shape)
        #cmatrix = confusion_matrix(y_test, np.rint(y_pred))
        #print ("confusion_matrix:", cmatrix, cmatrix.shape)

        print(f"{Colours.YELLOW.value}Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}{Colours.NORMAL.value}")

        # Write the same message to the "Output/Server_Evaluation.txt" file
        with open(f"Output/{sub_dir_name}/{test_directory_name}/Server_Evaluation.txt", "a") as f:
            f.write(f"{datetime.datetime.now()} - {server_round} : Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}\n")

            # Close the file
            f.close()

        np.save("y_pred-" + str(eval_count) + ".npy", y_pred)
        #np.save("cmatrix-" + str(eval_count) + ".npy", cmatrix)
        eval_count = eval_count + 1

        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
    #Flatten(input_shape=(79,1)),
    Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
    Dense(50, activation='relu'),
    Dense(25, activation='relu'),
    Dense(num_unique_classes, activation='softmax')
])


server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

scikit-learn 1.6.1.
flwr 1.18.0
numpy 1.26.4
tf 2.16.2


In [120]:
%%time
print (f"{Colours.YELLOW.value}\nDeploy simulation... Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier")
print (f"Number of Clients = {NUM_OF_CLIENTS}\n")
print (f"Writing output to: {sub_dir_name}/{test_directory_name}\n{Colours.NORMAL.value}")

# Output the same information to the Output/Run_details.txt file
with open(f"Output/{sub_dir_name}/{test_directory_name}/Run_details.txt", "a") as f:
    f.write(f"{datetime.datetime.now()} - Deploy simulation... Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier\n")
    f.write(f"{datetime.datetime.now()} - Number of Clients = {NUM_OF_CLIENTS}\n")

    # Write Original train_df size
    f.write(f"{datetime.datetime.now()} - Original train_df size: {train_df_shape}\n")

    # Write the training data split groups
    for i in range(len(fl_X_train)):
        f.write(f"{datetime.datetime.now()} - {i}: X Shape {fl_X_train[i].shape}, Y Shape {fl_y_train[i].shape}\n")

    # Write the testing data
    f.write(f"{datetime.datetime.now()} - X_test size: {X_test.shape}\n")
    f.write(f"{datetime.datetime.now()} - y_test size: {y_test.shape}\n")

# close the file
f.close()

start_time = datetime.datetime.now()

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

end_time = datetime.datetime.now()
print("Total time taken: ", end_time - start_time)

print (f"{Colours.YELLOW.value} SIMULATION COMPLETE. Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier")
print (f"Number of Clients = {NUM_OF_CLIENTS}{Colours.NORMAL.value}\n")

# Output the same information to the Output/Run_details.txt file
with open(f"Output/{sub_dir_name}/{test_directory_name}/Run_details.txt", "a") as f:
    f.write(f"{datetime.datetime.now()} - SIMULATION COMPLETE. Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier\n")
    f.write(f"{datetime.datetime.now()} - Total time taken: {end_time - start_time}\n")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout



Deploy simulation... Method = LEAVE_ONE_OUT - Binary (2) Classifier
Number of Clients = 10

Writing output to: train_size-94650/LEAVE_ONE_OUT_Classifier-2_Clients-10



Python(76195) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(76198) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(76199) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(76200) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(76201) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(76202) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(76203) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
2025-06-14 23:32:30,942	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 8.0, 'object_store_memory': 2147483648.0, 'node:127.0.0.1': 1.0, 'node:__internal_head__': 1.0, 'memory': 5777575936.0}
INFO :      Optimize your simulation with Flower VCE:

(ClientAppActor pid=76223) Client ID: 9
Server Evaluating... Evaluation Count:0
22667/22667 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.4293 - loss: 0.7293
22667/22667 ━━━━━━━━━━━━━━━━━━━━ 20s 898us/step


INFO :      initial parameters (loss, other metrics): 0.7292167544364929, {'accuracy': 0.4292775094509125}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9181 - loss: 0.3320  
(ClientAppActor pid=76220) Client 6 evaluation complete - Accuracy: 0.905830, Loss: 0.325438


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 707.11s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.03153235899433319
INFO :      		round 2: 0.024159282943336854
INFO :      		round 3: 0.023110072317942165
INFO :      		round 4: 0.03090807944377931
INFO :      		round 5: 0.022328264885467734
INFO :      		round 6: 0.02215894580341907
INFO :      		round 7: 0.043248771206373285
INFO :      		round 8: 0.03204369294033973
INFO :      		round 9: 0.030712737187991196
INFO :      		round 10: 0.022506470240415716
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.7292167544364929
INFO :      		round 1: 0.022168904542922974
INFO :      		round 2: 0.020841442048549652
INFO :      		round 3: 0.020374272018671036
INFO :      		round 4: 0.020134471356868744
INFO :      		round 5: 0.021449429914355278
INFO :      		round 6: 0.020852094516158104
INFO :      		round 7: 0

Total time taken:  0:13:30.871013
 SIMULATION COMPLETE. Method = LEAVE_ONE_OUT - Binary (2) Classifier
Number of Clients = 10

CPU times: user 11min 54s, sys: 1min 29s, total: 13min 23s
Wall time: 13min 30s


Python(76449) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9961 - loss: 0.0099
(ClientAppActor pid=76221) Client 1 evaluation complete - Accuracy: 0.995774, Loss: 0.010343



## 3.2.3 Huấn luyện mô hình Federated Learning và theo dõi kết quả

Phần này triển khai vòng lặp huấn luyện chính trong Federated Learning, nơi mô hình toàn cục được gửi tới từng client để huấn luyện cục bộ. Sau đó, các trọng số của client được tổng hợp lại tại server bằng thuật toán FedAvg để cập nhật mô hình toàn cục.


In [ ]:

# Khởi tạo mô hình toàn cục
global_model = build_model()

# Danh sách lưu kết quả accuracy để vẽ biểu đồ nếu cần
global_accuracies = []

for round_num in range(NUM_OF_ROUNDS):
    local_weights = []
    print(f"--- Round {round_num + 1} ---")

    for client_data in client_datasets:
        # Clone mô hình toàn cục cho client
        local_model = clone_model(global_model)
        local_model.set_weights(global_model.get_weights())
        local_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Huấn luyện mô hình trên dữ liệu cục bộ
        local_model.fit(client_data['x_train'], client_data['y_train'], epochs=1, verbose=0)
        local_weights.append(local_model.get_weights())

    # Áp dụng FedAvg để cập nhật mô hình toàn cục
    new_weights = FedAvg(local_weights)
    global_model.set_weights(new_weights)

    # Đánh giá mô hình toàn cục trên tập kiểm tra chung (global test set)
    loss, acc = global_model.evaluate(x_test_global, y_test_global, verbose=0)
    global_accuracies.append(acc)
    print(f"Global model - Loss: {loss:.4f} - Accuracy: {acc:.4f}")
